In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [ ]:
# Load data
youtube_df = pd.read_csv('USvideos.csv')
youtube_df.head()

In [ ]:
# Extract video category labels and ids from auxillary JSON file
with open("US_category_id.json", "r") as f:
    data = json.load(f)

# Create a dictionary mapping ids to lablels
categories = {}
for item in data['items']:
    id = int(item['id'])
    category = item['snippet']['title']
    categories[id] = category

print(categories)